# Introduction to SQL

---

## ✨ Structured Query Language (SQL)

---

### ❓ Why use SQL? Can't we use Python (Pandas)? 🐼

Using Pandas can be easier in many cases unless you're familiar with SQL. Many data science professionals will agree that Pandas is a more human-friendly method ([discussion here](https://datascience.stackexchange.com/questions/34357/why-do-people-prefer-pandas-to-sql)). You can perform the most common operations in SQL with Pandas ([comparison with SQL here](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html)).

![Kevin](https://github.com/bdi475/images/blob/main/lecture-notes/sql/kevin-malone-pandas-sql.jpg?raw=true)

Huh? When are you supposed to use SQL then? 🤨 You should use SQL when you have a large source of data on a remote location. Pandas works great... until you run out of memory (RAM). When you read data from a source, Pandas loads the entire data into memory by default. Pandas will fail if your data is too large.

Instead of loading all data into memory first, you can query a database system and retrieve only the information you need. This is almost always faster and more efficient.

---

## ⚡️ Database Management Systems (DBMS)

A database is a collection of data. A database management system is a software used to let users communicate with a database. There are many types of DBMS software. We will only cover relational database management systems (RDBMS). Many popular RDBMS exist, such as:

- PostgreSQL
- Oracle Database
- MySQL
- MariaDB (fork of MySQL)
- SQLite

We will use [SQLite](https://www.sqlite.org/), since it is the easiest one to set up.

---

### ❓ Why use databases? Can't we use CSV files?

This discussion is somewhat similar to why we use SQL instead of Pandas. Relational database management systems have existed since the 1970s. They are highly optimized even when your data size is humongous. Of course, the performance is dependent on the *query* you write even if you use an RDBMS.

![Boardroom Meeting](https://github.com/bdi475/images/blob/main/lecture-notes/sql/data-storage-boardroom-meeting.jpg?raw=true)

---

## 📗 CRUD Operations Exercise using a graphical tool

The most common operations when interacting with a database are often referred to as *CRUD* (Create, Read, Update, Delete).

![CRUD acronym](https://github.com/bdi475/images/blob/25153765dde6ade3d54a48b17825a3dc1923d17e/lecture-notes/sql/crud-abbreviation.png?raw=true)

Go to [https://sqliteonline.com/](https://sqliteonline.com/) and try out these SQL queries.

---
**CREATE**: Create `employees` table.

```sql
CREATE TABLE IF NOT EXISTS employees (
  emp_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT,
  salary REAL
);
```

---
**UPDATE**: Add employees information.

```sql
INSERT INTO employees (name, department, salary) VALUES ('Mia', 'HR', 140000.0);
INSERT INTO employees (name, department, salary) VALUES ('Michael', 'Marketing', 200000.0);
INSERT INTO employees (name, department, salary) VALUES ('Sean', 'HR', 250000.0);
INSERT INTO employees (name, department, salary) VALUES ('Dylan', 'Marketing', 205000.0);
```

---
**READ**: Select `emp_id` and `name` fields from `employees` table.

```sql
SELECT emp_id, name
FROM employees;
```

---
**READ**: Select `emp_id`, `name`, and `department` fields of HR employees from `employees` table.

```sql
SELECT emp_id, name, department
FROM employees
WHERE department = "HR";
```

---
**READ**: Select `emp_id`, `name`, and `salary` fields of Marketing employees sorted by `salary` (highest salary first) from `employees` table.

```sql
SELECT emp_id, name, salary
FROM employees
WHERE department = "Marketing"
ORDER BY salary DESC;
```

---
**READ**:  Select all rows and columns from `employees` table.

```sql
SELECT * FROM employees;
```

---
**DELETE**: Delete all HR employees information

```sql
DELETE FROM employees WHERE department = "HR";
```

---

## 📘 CRUD Operations using Jupyter notebook

▶️ First, run the code cell below to import modules used for **🧭 Check Your Work** sections and the autograder.

In [ ]:
import unittest
import base64
tc = unittest.TestCase()

#### 👇 Tasks

- ✔️ Import the following Python packages.
    1. `pandas`: Use alias `pd`.
    2. `numpy`: Use alias `np`.
    3. `sqlite3`: No alias

In [ ]:
### BEGIN SOLUTION



### END SOLUTION

#### 🧭 Check your work

In [ ]:
import sys
tc.assertTrue('pd' in globals(), 'Check whether you have correctly imported Pandas with an alias.')
tc.assertTrue('np' in globals(), 'Check whether you have correctly imported NumPy with an alias.')
tc.assertTrue('sqlite3' in globals(), 'Check whether you have correctly imported the sqlite3 package.')

---
### 🎯 Create a database file and a table

▶️ Run the code below to create a new SQLite database file named `airbnb.db` and create a table named `listings`.

In [ ]:
# Connect to airbnb.db database file
# If the file does not exist, create a new file
conn = sqlite3.connect('airbnb.db')

# Create a cursor
# A cursor enables users of a DBMS to traverse through the result set
# without having to retrieve all results at once
c = conn.cursor()

# Drop (delete) listings table if it already exists
c.execute('DROP TABLE IF EXISTS listings')
conn.commit()

# Define a query
# Triple quotes (''') denote multiline strings
create_table_query = '''
CREATE TABLE IF NOT EXISTS listings (
    neighbourhood TEXT,
    room_type TEXT,
    bathrooms REAL,
    bedrooms REAL,
    price REAL,
    number_of_reviews INTEGER,
    review_score REAL,
    is_superhost INT
)
'''

c.execute(create_table_query)
conn.commit()

conn.close()

---
### 🎯 Populate a database table from a CSV file

▶️ Run the code below to create a DataFrame named `df_listings`.

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
df_listings = pd.read_csv('https://github.com/UI-Deloitte-business-analytics-center/datasets/raw/main/chicago-airbnb-listings-small.csv')

display(df_listings.head(5))

▶️ Run the code below to populate the `listings` table. All data in `df_listings` will be inserted to the table.

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')
c = conn.cursor()

tables = list(pd.read_sql_query('SELECT * FROM sqlite_master WHERE type="table";', con=conn)['tbl_name'])
if 'listings' in tables:
    c.execute(f'DELETE FROM listings')
    conn.commit()

df_listings.to_sql(name='listings', index=False, con=conn, if_exists='append')

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
import glob
import os
user_db_files = glob.glob('*.db')

# Check if all files exist in the current directory
tc.assertTrue('airbnb.db' in user_db_files, f'Check if airbnb.db exists in {os.getcwd()}')

conn_checker = sqlite3.connect('airbnb.db')
table_to_check = 'listings'

# Check if table exists
user_tables = list(pd.read_sql_query('SELECT * FROM sqlite_master WHERE type="table";', con=conn_checker)['tbl_name'])
tc.assertTrue(table_to_check in user_tables, f'{table_to_check} does not exist in your NWT.db file!')

conn_checker.close()

---
### 🎯 Exercise 1: Select all rows and columns

#### 👇 Tasks

- ✔️ Write a query that selects all columns for all rows.
- ✔️ Store your query in a new variable named `query_select_all`.

In [ ]:
### BEGIN SOLUTION




### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_select_all, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCAqIEZST00gbGlzdGluZ3MK').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')

conn.close()

---
### 🎯 Exercise 2: Select only three columns for all rows

#### 👇 Tasks

- ✔️ Write a query that only the following 3 columns from the `listings` table:
    1. `bathrooms`
    2. `bedrooms`
    3. `price`
- ✔️ Store your query in a new variable named `query_select_three_columns`.

In [ ]:
### BEGIN SOLUTION




### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_select_three_columns, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCBiYXRocm9vbXMsIGJlZHJvb21zLCBwcmljZSBGUk9NIGxpc3RpbmdzCg==').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')

conn.close()

---
### 🎯 Exercise 3: Select all columns of expensive listings

#### 👇 Tasks

- ✔️ Write a query that selects all columns of listings with a daily price greater than $400 in the `listings` table:
- ✔️ Store your query in a new variable named `query_expensive_listings`.

In [ ]:
### BEGIN SOLUTION





### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_expensive_listings, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCAqCkZST00gTElTVElOR1MKV0hFUkUgcHJpY2UgPiA0MDAK').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')

conn.close()

---
### 🎯 Exercise 4: Select all columns of West Town listings

#### 👇 Tasks

- ✔️ Write a query that selects all columns of listings in the "West Town" neighbourhood.
    - `WHERE neighbourhood = 'West Town'`
- ✔️ Store your query in a new variable named `query_west_town_listings`.

In [ ]:
### BEGIN SOLUTION





### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_west_town_listings, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCAqCkZST00gTElTVElOR1MKV0hFUkUgbmVpZ2hib3VyaG9vZCA9ICdXZXN0IFRvd24nCg==').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')

conn.close()

---
### 🎯 Exercise 5: Sort all listings by price

#### 👇 Tasks

- ✔️ Write a query that selects all listings and sorts the result by price in ascending order.
- ✔️ Store your query in a new variable named `query_sort_by_price`.

In [ ]:
### BEGIN SOLUTION





### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_sort_by_price, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCAqCkZST00gTElTVElOR1MKT1JERVIgQlkgcHJpY2UK').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')

conn.close()

---
### 🎯 Exercise 6: Select only the price column and sort in descending order

#### 👇 Tasks

- ✔️ Write a query that selects only the price column and sorts the result in descending order.
- ✔️ Store your query in a new variable named `query_sort_by_price_desc`.

In [ ]:
### BEGIN SOLUTION





### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_sort_by_price_desc, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCBwcmljZQpGUk9NIExJU1RJTkdTCk9SREVSIEJZIHByaWNlIERFU0MK').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')
pd.testing.assert_series_equal(df_result['price'].reset_index(drop=True),
                               df_check['price'].reset_index(drop=True))

conn.close()

---
### 🎯 Exercise 7: A more complicated query

#### 👇 Tasks

- ✔️ Write a query that:
    - Selects `room_type`, `bathrooms`, `bedrooms`, `price`, and `number_of_reviews` columns (in the same order).
    - Where `price` is less than 100 **AND** the number of reviews is greather than `500`.
    - Sorts by `number_of_reviews` in descending order.
- ✔️ Store your query in a new variable named `query_final`.

In [ ]:
### BEGIN SOLUTION






### END SOLUTION

conn = sqlite3.connect('airbnb.db')

df_result = pd.read_sql_query(query_final, con=conn)
display(df_result)

conn.close()

#### 🧭 Check your work

In [ ]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn = sqlite3.connect('airbnb.db')

decoded_query = base64.b64decode(b'ClNFTEVDVCByb29tX3R5cGUsIGJhdGhyb29tcywgYmVkcm9vbXMsIHByaWNlLCBudW1iZXJfb2ZfcmV2aWV3cwpGUk9NIExJU1RJTkdTCldIRVJFIChwcmljZSA8IDEwMCkgQU5EIChudW1iZXJfb2ZfcmV2aWV3cyA+IDUwMCkKT1JERVIgQlkgbnVtYmVyX29mX3Jldmlld3MgREVTQwo=').decode()
df_check = pd.read_sql_query(decoded_query, con=conn)

tc.assertEqual(df_result.shape, df_check.shape, 'Number of rows and/or columns is different')
pd.testing.assert_frame_equal(df_result.reset_index(drop=True),
                               df_check.reset_index(drop=True))

conn.close()